In [ ]:
import matplotlib.pyplot as plt
import ROOT 
import numpy as np
import CanvasSetup as cSetup
import os,copy
import itertools as itrTools

In [ ]:
import ROOT 
import operator 
import array

from CanvasSetup import *

def getFitParams(hist0):
    mean  = hist0.GetMean()
    stdDev= hist0.GetStdDev()
    minX=mean-10*stdDev
    maxX=mean+10*stdDev
    hist0.Fit("gaus","Q0","",minX,maxX);
    f=hist0.GetFunction("gaus");
    if f:
        return [f.GetParameter(1),f.GetParameter(2),f]
    else:
        return [-1.0,1.0,None]
    
def saveTheDictionary(aCollection,fname=None,folder=None):
    outfile=folder
    if fname!=None and outfile==None:
        outfile=ROOT.TFile(fname,"RECREATE")
    outfile.cd()
    for key in aCollection:
#         print("Doing : ",key)
        if type(aCollection[key])==type({}):
            print("\tMakdig dir : ",key)
            inFolder=outfile.mkdir(key)
            saveTheDictionary(aCollection[key],None,inFolder)
        else:
            aCollection[key].Write()
    if fname!=None:
        outfile.Close()

colurWheel=[ 
	   ROOT.kBlack  ,
	   ROOT.kRed    ,
	   ROOT.kGreen  ,
       ROOT.kBlue   ,
	   ROOT.kMagenta ,
	   ROOT.kYellow ,
	   ROOT.kOrange ,
       ROOT.kSpring ,
	   ROOT.kCyan   ,
	   ROOT.kAzure   ,
	   ROOT.kTeal   ,
	   ROOT.kGray    ,
	   ROOT.kViolet ,
	   ROOT.kPink   ,
    ]

class Plot:
    def __init__(self, **args):
        self.name        = args.get("Name", "plot")
        self.legend      = args.get("Legend",None)
        self.histo       = args.get("Histo", None)
        self.fit         = args.get("Fit", None)
        self.markerColor = args.get("MarkerColor", ROOT.kBlue+2)
        self.markerStyle = args.get("MarkerStyle", 22)
        self.markerSize = args.get("MarkerSize", 5)
        self.lineColor   = args.get("LineColor", ROOT.kBlue+2)
        self.lineStyle   = args.get("LineStyle", 1)
        self.lineWidth   = args.get("LineWidth", 1)
        self.options   = args.get("Options", "")
        self.drawLegend   = args.get("DrawLegend",False)
        self.drawLine     = args.get("DrawLine", False)
        self.histo.SetName(self.name+"_histo")
        self.normalize =args.get("Normalize",False)
        self.scaleFactor =args.get("ScaleFactor",None)
        self.doFit =args.get("doFit",False)
        self.drawFit =args.get("doFit",False)
        self.verbose =args.get("verbose",True)

class PlotCanvas:
    def __init__(self, **args):
        self.name  = ""
        self.plots = []
        self.rates = []
        self.plotDir =  args.get("prefix","plots/")
        self.yRange = (0.0, 1.0)
        self.xRange = (0, 100)
        self.yRange2 = (0.0, 1.0)
        self.xTitle = args.get("xTitle","E_{T}^{e offl} [GeV]")
        self.yTitle = args.get("yTitle","a.u")
        self.yTitle2 = args.get("yTitle","a.u")
        self.legendPosition = (0.4,0.2,0.9,0.6)
        self.descPosition = (0.6,0.58)
        self.desc = args.get("desc",["L1 Efficiency"])
        self.logx = args.get("logx",False)
        self.logy = args.get("logy",False)
        self.logy2 = args.get("logy2",False)
        self.canvasParams = args.get("canvasPars",getDefaultCanvas())    
        self.setPlotStyle()

    def addRate(self, rate):
        self.rates.append(rate)
    def addPlot(self, plot):
        self.plots.append(plot)
    def clearPlots(self):
        self.plots=[]


    def plot(self):
        xmax=0.0
        xmin=1e9
        xmax_=1e9
        xmin_=0.0
        ymax=0.0
        ymin=1e9
        ymax_=1e9
        ymin_=0.0
        for plot in self.plots:
            histo = plot.histo
            print("Scale Factor  : before  = ",plot.scaleFactor)    
            if histo.Integral() ==0 :
                continue
            if plot.normalize:
                x= histo.Integral()
                if x==0:
                    x=1.0
                if plot.scaleFactor==None:
                    plot.scaleFactor=1.0/( x)
                else:
                    plot.scaleFactor/=(x)
                print("Integral = = ",1.0/(x))    
            if plot.scaleFactor!=None:
                histo.Scale(plot.scaleFactor)
            print("Scaled the histogram by a factor of : ",plot.scaleFactor)
            print("             Integral after Scaling : ",histo.Integral())
            
            if self.yRange[1]=='auto':
                ymax_=histo.GetMaximum()
                if ymax < ymax_*1.1:
                    ymax=ymax_*1.1
            if self.yRange[0]=='auto': 
                ymin_=histo.GetMinimum()
                if ymin > ymin_*0.8:
                    ymin=ymin_*0.8
            
            if self.xRange[1]=='auto':
                xmax_=histo.GetXaxis().GetBinCenter( histo.GetNbinsX() ) + 0.5*histo.GetBinWidth(histo.GetNbinsX())
                print(xmax_,xmax)
                if abs(xmax) < abs(xmax_):
                    xmax=xmax_
            if self.xRange[0]=='auto': 
                xmin_=histo.GetXaxis().GetBinCenter(1) - 0.5*histo.GetBinWidth(1)
                print(xmin_,xmin)
                if abs(xmin) > abs(xmin_):
                    xmin=xmin_
        
        if self.yRange[0]=='auto':
            self.yRange=(ymin,self.yRange[1])
        if self.yRange[1]=='auto':
            self.yRange=(self.yRange[0],ymax)
        if self.xRange[0]=='auto':
            self.xRange=(xmin,self.xRange[1])
        if self.xRange[1]=='auto':
            self.xRange=(self.xRange[0],xmax)

#         print("Setting  X-RANGE  : ",self.xRange)
#         print("Setting  Y-RANGE  : ",self.yRange)
        
#         print("xmax xmin : ",xmin,xmax)
        canvas = ROOT.TCanvas("c_"+self.name, self.name, 700, 800)
        canvas.SetGrid()
        hDummy = ROOT.TH1F("hDummy_"+self.name, self.name, 1, self.xRange[0],self.xRange[1])

        #print("ymin_,ymax_",ymin_,ymax_,"ymin , ymax - > " ,ymin,ymax)
            #print( "\tY : Min - > Max  : " ,histo.GetMinimum()," -> ", histo.GetMaximum())
        hDummy.SetAxisRange(self.yRange[0], self.yRange[1], "Y")
        hDummy.SetXTitle(self.xTitle)
        hDummy.SetYTitle(self.yTitle)
        hDummy.Draw("C")
        if self.logx : canvas.SetLogx()
        if self.logy : canvas.SetLogy()
        self.logz= True
        if self.logz : canvas.SetLogz()
        
        
        CMSbox=self.getCMSBox()
        extraTextBox=self.getExtraTextBox()
        lumibox=self.getLumiBox()


        selbox=[]
        n=0
        for inx in range(len(self.desc)):
            selbox.append(ROOT.TLatex  (self.descPosition[0], self.descPosition[1] -n*0.04, self.desc[n]))
            selbox[-1].SetNDC()
            selbox[-1].SetTextSize(0.035)
            selbox[-1].SetTextFont(12)
            selbox[-1].SetTextAlign(13)
            n+=1
        for plot in self.plots:
            if(plot.doFit):
                histo = plot.histo
                fitP  = getFitParams(histo)
                strFit='('+'{0:0.3f}'.format(fitP[0])+ ' , '+'{0:0.3f}'.format(fitP[1]) +')'
                selbox.append(ROOT.TLatex  (self.descPosition[0], self.descPosition[1] -n*0.04, strFit))
                selbox[-1].SetNDC()
                selbox[-1].SetTextSize(0.035)
                selbox[-1].SetTextFont(12)
                selbox[-1].SetTextColor(plot.lineColor)
                selbox[-1].SetTextAlign(13)
                if plot.drawFit and  fitP[2]:
                    fitP[2].SetLineStyle(plot.lineStyle)
                    fitP[2].SetLineWidth(2)
                    fitP[2].SetLineColor(plot.lineColor)
                    fitP[2].Draw("same")
                n+=1

        #Line legend
        legend = ROOT.TLegend(self.legendPosition[0],self.legendPosition[1],self.legendPosition[2],self.legendPosition[3])
        legend.SetTextFont(42)
        legend.SetFillColor(0)

        for plot in self.plots:
            histo = plot.histo
            print("Ploting hist : ",histo.GetName())
            if plot.verbose:
                print( "\tX : Min - > Max  : " ,histo.GetBinCenter(1)," -> ", histo.GetBinCenter(histo.GetNbinsX())," [ ",histo.GetBinCenter(2)-histo.GetBinCenter(1)," ]" )
                print( "\tY : Min - > Max  : " ,histo.GetMinimum()," -> ", histo.GetMaximum())
                
            histo.SetMarkerStyle(plot.markerStyle)
            histo.SetMarkerSize(plot.markerSize)
            histo.SetMarkerColor(plot.markerColor)
            histo.SetLineStyle(plot.lineStyle)
            histo.SetLineWidth(plot.lineWidth)
            histo.SetLineColor(plot.lineColor)
            #histo.Draw("same e")
#             print(plot.options)
            histo.Draw("same "+plot.options)
#             print( histo.GetOption())

            if(plot.legend):
                legend.AddEntry(histo, plot.legend, "pe")
                legend.SetTextSize(0.025) 
            if(plot.drawLegend) :  legend.Draw()
        
        scale = (self.yRange[1] -self.yRange[0])/(self.yRange2[1] - self.yRange2[0])
        y20=self.yRange2[0]

        if(self.logy2):
            scale = (self.yRange[1] -self.yRange[0])/(ROOT.log(self.yRange2[1]) - ROOT.log(self.yRange2[0]) )
            y20 = ROOT.log(self.yRange2[0])


        #####   begin :            For plotting a graph in the Second Axis         #####

        if len(self.rates) >0 :
            axis=None
            canvas.SetRightMargin(0.18);
            if(self.logy2):
                axis = ROOT.TGaxis(self.xRange[1],self.yRange[0],self.xRange[1], self.yRange[1],self.yRange2[0],self.yRange2[1],510,"+LG")
            else :
                axis = ROOT.TGaxis(self.xRange[1],self.yRange[0],self.xRange[1], self.yRange[1],self.yRange2[0],self.yRange2[1],510,"+L")
            axis.SetLineColor(ROOT.kRed);
            axis.SetTextColor(ROOT.kRed);
            axis.SetTitle(self.yTitle2)
            axis.SetTitleColor(ROOT.kBlack)
            axis.SetTitleOffset(1.2)
            axis.Draw();

        for rate in self.rates:
            histo = rate.histo
            if self.logy2:
                for i in range(histo.GetNbinsX()):
                    content=histo.GetBinContent(i+1)
                    scaledContent = ( ROOT.log(content + 1e-20 )  - y20 )* scale
                    histo.SetBinContent(i+1,scaledContent)
                    #print("\t ",content," -> ",scaledContent,"  = ( ",ROOT.log(content + 1e-20),"  - ",y20," )* ",scale)
            else:
                for i in range(histo.GetNbinsX()):
                    content=histo.GetBinContent(i+1)
                    scaledContent = (content  - y20 )* scale
                    histo.SetBinContent(i+1,scaledContent)
            
            histo.SetMarkerStyle(rate.markerStyle)
            histo.SetMarkerColor(rate.markerColor)
            histo.SetLineColor(rate.markerColor)
            histo.Draw("pl same "+ rate.options)
            legend.AddEntry(histo, rate.legend, "pe")
            legend.SetTextSize(0.025) 
            if(rate.drawLegend) :  legend.Draw()
        
        #####   end :            For plotting a graph in the Second Axis         #####
            
        CMSbox.Draw()
        extraTextBox.Draw()
        lumibox.Draw()
        for selb in selbox:
            selb.Draw()
        
#         canvas.Print(self.plotDir+"/"+self.name+".pdf", "pdf")
        canvas.Print(self.plotDir+"/"+self.name+".png", "png")

        return canvas

    def getCMSBox(self):
        par=self.canvasParams["CMS"]
        CMSbox       = ROOT.TLatex  (par["xpos"],par["ypos"], par["text"])
        CMSbox.SetNDC()
        CMSbox.SetTextSize(par["textSize"])
        CMSbox.SetTextFont(par["textFont"])
        CMSbox.SetTextColor(ROOT.kBlack)
        CMSbox.SetTextAlign(13) 
        return CMSbox
   
    def getExtraTextBox(self):
        par=self.canvasParams["TAG"]
        extraTextBox = ROOT.TLatex  (par["xpos"], par["ypos"], par["text"])
        extraTextBox.SetNDC()
        extraTextBox.SetTextSize(par["textSize"])
        extraTextBox.SetTextFont(par["textFont"])
        extraTextBox.SetTextColor(ROOT.kBlack)
        extraTextBox.SetTextAlign(13)
        
        return extraTextBox

    def getLumiBox(self):
        par=self.canvasParams["EnLumi"]
        lumibox = ROOT.TLatex  ( par["xpos"], par["ypos"], par["text"])
        lumibox.SetNDC()
        lumibox.SetTextAlign(31)
        lumibox.SetTextSize(par["textSize"])
        lumibox.SetTextFont(par["textFont"])
        lumibox.SetTextColor(ROOT.kBlack)
        
        return lumibox
        
    def setPlotStyle(self):
        ROOT.gROOT.SetStyle("Plain")
        ROOT.gStyle.SetOptStat()
        ROOT.gStyle.SetOptFit(0)
        ROOT.gStyle.SetOptTitle(0)
        ROOT.gStyle.SetFrameLineWidth(2)
        ROOT.gStyle.SetPadBottomMargin(0.13)
        ROOT.gStyle.SetPadLeftMargin(0.15)
        ROOT.gStyle.SetPadTopMargin(0.06)
        ROOT.gStyle.SetPadRightMargin(0.05)

        ROOT.gStyle.SetLabelFont(42,"X")
        ROOT.gStyle.SetLabelFont(42,"Y")
        ROOT.gStyle.SetLabelSize(0.04,"X")
        ROOT.gStyle.SetLabelSize(0.04,"Y")
        ROOT.gStyle.SetLabelOffset(0.01,"Y")
        ROOT.gStyle.SetTickLength(0.02,"X")
        ROOT.gStyle.SetTickLength(0.02,"Y")
        ROOT.gStyle.SetLineWidth(2)
        ROOT.gStyle.SetTickLength(0.02 ,"Z")

        ROOT.gStyle.SetTitleSize(0.1)
        ROOT.gStyle.SetTitleFont(42,"X")
        ROOT.gStyle.SetTitleFont(42,"Y")
        ROOT.gStyle.SetTitleSize(0.05,"X")
        ROOT.gStyle.SetTitleSize(0.05,"Y")
        ROOT.gStyle.SetTitleOffset(1.1,"X")
        ROOT.gStyle.SetTitleOffset(1.4,"Y")
        ROOT.gStyle.SetOptStat(0)
        ROOT.gStyle.SetPalette(1)
        ROOT.gStyle.SetPaintTextFormat("3.2f")
        ROOT.gROOT.ForceStyle()

col=[ 0,ROOT.kBlack,ROOT.kRed,ROOT.kBlue,ROOT.kGreen,6,7,8,9,10,11,12,13,14,15  ]

def getDefaultPlotParams(col=1,marker=22,markerSize=1):
    return {'Legend':"Data : 2018, Unpacked",
                          'MarkerColor' : col,
                          'LineColor':col,
                          'LineWidth':2,
                          'LineStyle':9,
                          'MarkerStyle':marker,
                          'MarkerSize':markerSize,
                          'DrawLegend':True,
                          'Option':'pc'
            } 

def getDefaultPlot(prefix='plots/',name='defaultName',cPars=cSetup.getCanvasParams('GEN')):
#     plot=putil.PlotCanvas(prefix=prefix,canvasPars=cPars)
    plot=PlotCanvas(prefix=prefix,canvasPars=cPars)
    
    plot.name   = name
    plot.xRange = (3.0,50.0)  
    plot.yRange = (0.0,1.02) 
    plot.yTitle = "#epsilon"  
    plot.xTitle = "E_{T}^{offl.}"
    plot.desc   = ["L1 EG Isolation"] 
    plot.legendPosition = (0.6,0.5,0.90,0.65)
    plot.descPosition   = (0.6,0.75)     
    plot.logx = False
    plot.logy = False
    return plot

def getCanvasParams(tag,lumi="-1 fb"):
    cPars=getDefaultCanvas()
    print("Canvas Tag : ", tag )
    if tag=="2018Full":
        cPars["EnLumi"]["xpos"]       =  0.961
        cPars["EnLumi"]["text"]       = "SingleEG 2018 (58.9fb^{-1}) 13 TeV"
        return cPars
    if tag=="Run3MC":
        cPars["EnLumi"]["xpos"]       =  0.961
        cPars["EnLumi"]["text"]       = "Run 3 MC, 13,6 TeV"
        return cPars
    if tag=="Run3Data":
        cPars["EnLumi"]["xpos"]       =  0.961
        cPars["EnLumi"]["text"]       = "Run 3 Egamma ( "+lumi+")"
        return cPars
    if tag=="Run3DataX":
        cPars["EnLumi"]["xpos"]       =  0.961
        cPars["EnLumi"]["text"]       = "Run 3 Egamma"
        return cPars
    if tag=="Run3DataZB":
        cPars["EnLumi"]["xpos"]       =  0.961
        cPars["EnLumi"]["text"]       = "ZeroBias( "+lumi+")"
        return cPars
     
    return cPars

def getTheObjectsFromFile(aFile):
    histStore={}
    for key in aFile.GetListOfKeys():
        kk=key.GetName()
        curObj=aFile.Get(kk)
        if curObj.Class_Name()=='TDirectoryFile':
            histStore[kk]=getTheObjectsFromFile(curObj)
        else:
            histStore[kk]=curObj       
    return histStore


## Plotting

In [ ]:
metadata=None
with open('data/fnames.json', 'r') as fp:
    metadata = json.load(fp)
    
fnames=metadata['fnames']   
legend=metadata['legends']    
colours={
    'dataRun3_unpacked'      : ROOT.kBlack ,
    'dataR3Unpaked_preCalib' : ROOT.kCyan ,
    'dataR3Unpaked_postCalib': ROOT.kGreen+2 ,
    'run3MC'                 : ROOT.kBlue ,
    'run3MCVsGenEt'                 : ROOT.kRed+1 ,
    'data2018'               : ROOT.kRed+1 ,
    'run3MCWithGenEt': ROOT.kMagenta ,
    'run3MCWithGenEtVsGenEt':  ROOT.kOrange +1,
    'run3MCWithGenEtVsGenEtBremSafe' : ROOT.kCyan+2,
    'run3MCWithGenEtVsGenEtZee' : ROOT.kGreen+1,
    'run3MCWithGenEtVsGenEtZDau' : ROOT.kBlack,
    'run3MCWithZDauGenEtVsGenEt' : ROOT.kGreen+1,
    'run3MCWithZDauGenEtVsGenEtBremSafe' : ROOT.kBlue,
    'run3MCWithZDauGenEtVsSCEt' : ROOT.kRed ,
    'run3DataWithHybridVsPFEt' : ROOT.kRed ,
    'run3DataWithHybridVsSCEt' : ROOT.kMagenta ,
    'run3DataDefaultVsPFEt' : ROOT.kBlue+2 ,
    'run3DataDefaultVsSCEt' : ROOT.kMagenta+2 ,
    'run3MCWithZDauGenEtVsPFEt' : ROOT.kGreen+2,
    'run3MCWithDefaultGenEtVsSCEt' : ROOT.kRed, #     'run3MCWithDefaultVsSCEt',
    'run3MCWithDefaultGenEtVsPFEt' : ROOT.kRed, #     'run3MCWithDefaultVsSCEt',
    'run3MCWithDefaultGenEtVsGenEt' : ROOT.kRed ,
    
    'run3DataDefaultVsSCEt' : ROOT.kRed,
    'run3DataDefaultVsPFEt' : ROOT.kRed,
    
    'run3DataWithHybridVsPFEt' : ROOT.kGreen+2,
    'run3DataWithHybridVsSCEt' : ROOT.kGreen+2,
    
    'run3MCWithZDauGenEtVsGenEt' : ROOT.kBlue,# todo
    'run3MCWithZDauGenEtVsPFEt'  : ROOT.kBlue,# todo
    'run3MCWithZDauGenEtVsSCEt'  : ROOT.kBlue,# todo
    
    'run3MCWithDefaultGenEtVsSCEt'  : ROOT.kMagenta, #     'run3MCWithDefaultVsSCEt',
    'run3MCWithDefaultGenEtVsPFEt'  : ROOT.kMagenta, #     'run3MCWithDefaultVsSCEt',
    'run3MCWithDefaultGenEtVsGenEt' : ROOT.kMagenta #     'run3MCWithDefauktVsGenEt',
    
}

In [ ]:
filesToLoad=[
    'dataR3Unpaked_postCalib',
    'data2018',
    
    'run3Data2022EWithHybridV3OldIsoVsSCEt',
    'run3Data2022EWithHybridV3OldIsoVsPFEt',
#     'run3Data2022EWithHybridV3NewIsoVsSCEt',
#     'run3Data2022EWithHybridV3NewIsoVsPFEt',
    
    'run3Data2022EDefaultVsSCEt',
    'run3Data2022EDefaultVsPFEt',
#     'run3Data2022EDefaultNewIsoVsSCEt',
#     'run3Data2022EDefaultNewIsoVsPFEt',
    
    'run3MCWithZDauGenEtVsGenEt',# todo
    'run3MCWithZDauGenEtVsPFEt' ,# todo
    'run3MCWithZDauGenEtVsSCEt' ,# todo
    
    'run3MCWithHybridV3VsGenEt',
    'run3MCWithHybridV3VsPFEt' ,
    'run3MCWithHybridV3VsSCEt' ,    
    
    'run3MCWithDefaultVsSCEt' , #     'run3MCWithDefaultVsSCEt',
    'run3MCWithDefaultVsPFEt' , #     'run3MCWithDefaultVsSCEt',
    'run3MCWithDefaultVsGenEt', #     'run3MCWithDefauktVsGenEt',
]

In [ ]:
histStore={}
histFileStore={}
for tag in filesToLoad:
    if tag in histFileStore:
        histFileStore[tag].Close()
    histFileStore[tag]=ROOT.TFile(fnames[tag],'READ')
    print("loading  tag ",tag)
    if tag=='data2018':
        histStore[tag]=getTheObjectsFromFile(histFileStore[tag])
    else:
        histStore[tag]=getTheObjectsFromFile(histFileStore[tag])['0']

In [ ]:
# for i in histStore['run3MCWithZDauGenEtVsPFEt']:
#     if 'Scale' in i:
#         print(i)
# #     print(i)
# #     if 'L1EtResolutionInEta-0.20To0.00'==i:
# #         print(" GGGG")

In [ ]:
if False: # vsGENEt : Hybrid LUT Validation : GetET full vs HybridEt
    tagsToPlot=[
        'run3MCWithZDauGenEtVsGenEt',
        'run3MCWithHybridV3VsGenEt',
    ]   
    colours={
        'run3MCWithZDauGenEtVsGenEt' : ROOT.kRed,
        'run3MCWithHybridV3VsGenEt'  : ROOT.kBlue,
    }
    legend={
        "run3MCWithZDauGenEtVsGenEt"  : "GenEt. Calib.,MC",
        "run3MCWithHybridV3VsGenEt"   : "Hybrid Calib.,MC",
    }
       
if True: # vsPFEt  : Hybrid LUT Validation : GetET full vs HybridEt
    tagsToPlot=[
        'run3Data2022EWithHybridV3OldIsoVsPFEt',
        'run3Data2022EDefaultVsPFEt',
    ]   
    colours={
        'run3Data2022EWithHybridV3OldIsoVsPFEt' : ROOT.kRed,
        'run3Data2022EDefaultVsPFEt'  : ROOT.kBlue,
    }
    legend={
        "run3Data2022EWithHybridV3OldIsoVsPFEt"  : "Hybrid Calib.",
        "run3Data2022EDefaultVsPFEt"   : "Default Calib.",
    }
       

if False: # Run3 MC Default vs GenEt calib
    tagsToPlot=[
        'run3MCWithDefaultGenEtVsPFEt',
        'run3MCWithZDauGenEtVsPFEt',
    ]    

if False:# old Iso in data 
    tagsToPlot=[
        'run3Data2022EWithHybridVsPFEt',
        'run3Data2022EDefaultVsPFEt',
    ]   
    colours={
        'run3Data2022EWithHybridVsPFEt' : ROOT.kRed,
        'run3Data2022EDefaultVsPFEt' : ROOT.kBlue,
    }

if False: # old Iso in data vs MC
    tagsToPlot=[
        'run3MCWithDefaultVsPFEt',
        'run3MCWithHybridV3VsPFEt',
        'run3Data2022EWithHybridV3NewIsoVsPFEt',
        'run3Data2022EDefaultVsPFEt',
    ]   
    colours={
        'run3MCWithDefaultVsPFEt' : ROOT.kCyan+1 ,
        'run3MCWithHybridV3VsPFEt': ROOT.kMagenta,
        'run3Data2022EWithHybridV3NewIsoVsPFEt' : ROOT.kRed,
        'run3Data2022EDefaultVsPFEt' : ROOT.kBlue,
    }
    legend={
        "run3MCWithDefaultVsPFEt":  "Default Calib., MC",
        "run3MCWithHybridV3VsPFEt"   :  "Hybrid Calib., MC",
        "run3Data2022EWithHybridV3NewIsoVsPFEt": "Hybrid Calib.,Data",
        "run3Data2022EDefaultVsPFEt"   : "Default Calib.,Data",
    }
        
        
if False: # old Iso in data vs new iso in Dats
    tagsToPlot=[
        'run3Data2022EWithHybridVsPFEt',
        'run3Data2022EDefaultVsPFEt',
        'run3Data2022EWithHybridNewIsoVsPFEt',
        'run3Data2022EDefaultNewIsoVsPFEt',
    ]   
    colours={
        'run3Data2022EWithHybridVsPFEt' : ROOT.kCyan+1,
        'run3Data2022EDefaultVsPFEt' : ROOT.kMagenta,
        'run3Data2022EWithHybridNewIsoVsPFEt' : ROOT.kRed,
        'run3Data2022EDefaultNewIsoVsPFEt' : ROOT.kBlue,
    }
    legend={
        "run3Data2022EWithHybridVsPFEt": "Hybrid Calib. , Old Iso",
        "run3Data2022EDefaultVsPFEt"   : "Default Calib. , Old Iso",
        "run3Data2022EWithHybridNewIsoVsPFEt": "Hybrid Calib. , New Iso",
        "run3Data2022EDefaultNewIsoVsPFEt"   : "Default Calib. , New Iso"
    }
    legends=legend
        
if False: # new Iso in data vs MC
    tagsToPlot=[
        'run3MCWithDefaultGenEtVsPFEt',
        'run3MCWithZDauGenEtVsPFEt',
        'run3Data2022EWithHybridNewIsoVsPFEt',
        'run3Data2022EDefaultNewIsoVsPFEt',
    ]   
    colours={
        'run3MCWithDefaultGenEtVsPFEt' : ROOT.kCyan+1 ,
        'run3MCWithZDauGenEtVsPFEt': ROOT.kMagenta,
        'run3Data2022EWithHybridNewIsoVsPFEt' : ROOT.kRed,
        'run3Data2022EDefaultNewIsoVsPFEt' : ROOT.kBlue,
    }
    legend={
        "run3MCWithDefaultGenEtVsPFEt": "Default Calib., MC",
        "run3MCWithZDauGenEtVsPFEt"   : "GenEt Calib., MC",
        "run3Data2022EWithHybridNewIsoVsPFEt": "Hybrid Calib.,Data",
        "run3Data2022EDefaultNewIsoVsPFEt"   : "Default Calib.,Data",
    }
        
    
baseCanvas='Run3DataX'
varToChose='Offline'
# varToChose='GEN'
# varToChose='SC'

prefix='results/plots/v7/data2022E/hybridV3/'
prefix+=varToChose+'/'
lumi='4.35 /fb'
if not os.path.exists(prefix):
    os.system('mkdir -p '+prefix)

## Gen Et scale vs Gen Et

In [ ]:
if False:  # Inclusive Et barrel Ecap SCEt
    plots=[]
    cPars=getCanvasParams(baseCanvas,lumi)
    rBin=1
    
    plot=getDefaultPlot(prefix=prefix,name='scEtVsdEtScaleBarrel',cPars=cPars) 
    plot.legendPosition = (0.23,0.73,0.86,0.85)
    plot.descPosition   = (0.25,0.75)
    plot.desc =[" "]
    plot.yTitle = "E_{T}^{L1} / E_{T}^{SC}"  
    plot.xTitle = "E_{T}^{GEN}"
    plot.xRange = (0.,120.0) 
    plot.logx = False
    plot.yRange = (0.8,2.14) 
    plots.append(plot) 

    i=2
    for tag in tagsToPlot:
        hname='EtVsdEtScaleBarrel'
        print(tag)
        hist=histStore[tag][hname].Clone()
    #         hist.Rebin(rBin)
        pparams=getDefaultPlotParams(col=i,marker=22)
        pparams['Legend']='Barrel , '+legend[tag]
        pparams['MarkerColor']= colours[tag]; pparams['LineColor'] = colours[tag] ;i+=1;
        pparams['Options']='HIST'
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor =None
        aplot.doFit=False ; aplot.drawFit=False
        aplot.verbose= False
        plots[-1].addPlot(aplot)


    plot=getDefaultPlot(prefix=prefix,name='scEtVsdEtScaleECap',cPars=cPars)  
    plot.legendPosition = (0.23,0.73,0.86,0.85)
    plot.descPosition   = (0.25,0.75)
    plot.desc =[" "]
    plot.yTitle = "E_{T}^{L1} / E_{T}^{SC}"  
    plot.xTitle = "E_{T}^{GEN}"
    plot.xRange = (0.8,120) 
    plot.logx = False
    plot.yRange = (0.8,2.14) 
    plots.append(plot) 
    
    i=4
    hname='EtVsdEtScaleECap'
    for tag in tagsToPlot:
        hist=histStore[tag][hname].Clone()
    #         hist.Rebin(rBin)
        pparams=getDefaultPlotParams(col=i,marker=22)
        pparams['Legend']='Endcap , '+legend[tag]
        pparams['MarkerColor']= colours[tag]; pparams['LineColor'] = colours[tag] ;i+=1;
        pparams['Options']='HIST'
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor =None
        aplot.doFit=False ; aplot.drawFit=False
        aplot.verbose= False
        plots[-1].addPlot(aplot)

    canvas = []
    for plot in plots:
        canvas.append(plot.plot())
    

In [ ]:
xAxisLbl=varToChose
if True:  # Inclusive Et barrel Ecap vs Gen Et
    plots=[]
    cPars=getCanvasParams(baseCanvas,lumi)
    rBin=1
    
    plot=getDefaultPlot(prefix=prefix,name='EtVsdEtScaleBarrel_'+xAxisLbl,cPars=cPars) 
    plot.legendPosition = (0.45,0.75,0.89,0.90)
    plot.descPosition   = (0.55,0.70)
    plot.desc =["Barrel"]
    plot.yTitle = "E_{T}^{L1} / E_{T}^{"+xAxisLbl+"}"  
    plot.xTitle = "E_{T}^{"+xAxisLbl+"}"
    plot.xRange = (0.,150.0) 
    plot.logx = False
    plot.yRange = (0.7,1.2) 
    plots.append(plot) 
    

    i=2
    for tag in tagsToPlot:
        hname='EtVsdEtScaleBarrel'
        hist=histStore[tag][hname].Clone()
    #         hist.Rebin(rBin)
        pparams=getDefaultPlotParams(col=i,marker=22)
        pparams['Legend']=legend[tag]
        pparams['MarkerColor']= colours[tag]; pparams['LineColor'] = colours[tag] ;i+=1;
        pparams['MarkerStyle']=8
        pparams['Options']='HIST pe'
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor =None
        aplot.drawLegend = True
        aplot.doFit=False ; aplot.drawFit=False
        aplot.verbose= False
        plots[-1].addPlot(aplot)


    plot=getDefaultPlot(prefix=prefix,name='EtVsdEtScaleECap_'+xAxisLbl,cPars=cPars)   
    plot.legendPosition = (0.45,0.75,0.89,0.90)
    plot.descPosition   = (0.55,0.70)
    plot.desc =["Endcap"]
    plot.yTitle = "E_{T}^{L1} / E_{T}^{"+xAxisLbl+"}"  
    plot.xTitle = "E_{T}^{"+xAxisLbl+"}"
    plot.xRange = (0.0,150) 
    plot.logx = False
    plot.yRange = (0.7,1.2) 
    plots.append(plot) 
    
    i=4
    hname='EtVsdEtScaleECap'
    for tag in tagsToPlot:
        hist=histStore[tag][hname].Clone()
    #         hist.Rebin(rBin)
        pparams=getDefaultPlotParams(col=i,marker=22)
        pparams['Legend']=legend[tag]
        pparams['MarkerColor']= colours[tag]; pparams['LineColor'] = colours[tag] ;i+=1;
        pparams['MarkerStyle']=8
        pparams['Options']='HIST pe'
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor =None
        aplot.doFit=False ; aplot.drawFit=False
        aplot.drawLegend = True
        aplot.verbose= False
        plots[-1].addPlot(aplot)

    for tag in tagsToPlot:
        plot=getDefaultPlot(prefix=prefix,name='EtVsdEtScale_'+tag+'_'+xAxisLbl,cPars=cPars)   
        plot.legendPosition = (0.45,0.75,0.89,0.90)
        plot.descPosition   = (0.55,0.70)
        plot.desc =[legend[tag]]
        plot.yTitle = "E_{T}^{L1} / E_{T}^{"+xAxisLbl+"}"  
        plot.xTitle = "E_{T}^{"+xAxisLbl+"}"
        plot.xRange = (0.0,150) 
        plot.logx = False
        plot.yRange = (0.7,1.2) 
        plots.append(plot) 
        cc={'EtVsdEtScaleBarrel':ROOT.kRed,'EtVsdEtScaleECap':ROOT.kBlue}
        ll={'EtVsdEtScaleBarrel':'Barrel','EtVsdEtScaleECap':'Endcap'}
        for hname in ['EtVsdEtScaleBarrel','EtVsdEtScaleECap']:
            hist=histStore[tag][hname].Clone()
            pparams=getDefaultPlotParams(col=i,marker=22)
            pparams['Legend']=ll[hname]
            pparams['MarkerColor']= cc[hname]; pparams['LineColor'] = cc[hname] ;i+=1;
            pparams['MarkerStyle']=8
            pparams['Options']='HIST pe'
            aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
            aplot.normalize = False ;aplot.scaleFactor =None
            aplot.doFit=False ; aplot.drawFit=False
            aplot.drawLegend = True
            aplot.verbose= False
            plots[-1].addPlot(aplot)
        
    canvas = []
    for plot in plots:
        canvas.append(plot.plot())
    

### OccupancyPlots

In [ ]:
if False:  # 2D plot of occupancy
    prefixH=prefix+'/occupancyPlots/'
    if not os.path.exists(prefixH):
        os.system('mkdir -p '+prefixH)
    for tag in tagsToPlot:
        for hname in histStore[tag]:
            if 'ocuppancy' not in hname:
                continue
            plots=[]
            cPars=getCanvasParams('RUN3DataX')
            rBin=1
            plot=getDefaultPlot(prefix=prefix,name=hname+tag,cPars=cPars) 
            plot.legendPosition = (0.65,0.78,0.9,0.90)
            plot.descPosition   = (0.60,0.85)
            plot.desc =['']
            plot.yTitle = "a.u"  
            plot.xTitle = "mass [GeV]"
            plot.xRange = (0.0,200.0) 
            plot.yRange = (0.0,200.0) 
            plots.append(plot)

            canvas = ROOT.TCanvas("c", "", 700,700)
            canvas.SetGrid()
            plot.setPlotStyle()

            CMSbox=plot.getCMSBox()  
            extraTextBox=plot.getExtraTextBox()
            lumibox=plot.getLumiBox()

            selbox=[]
            n=0
            for inx in range(len(plot.desc)):
                selbox.append(ROOT.TLatex  (plot.descPosition[0], plot.descPosition[1] -n*0.04, plot.desc[n]))
                selbox[-1].SetNDC()
                selbox[-1].SetTextSize(0.035)
                selbox[-1].SetTextFont(12)
                selbox[-1].SetTextAlign(13)
                n+=1
            for i in range(len(selbox)):
                selbox[i].Draw()
            CMSbox.Draw()
            extraTextBox.Draw()
            lumibox.Draw()

            hist=histStore[tag][hname].Clone()
            hist.Draw('colz')
            x=hist.GetXaxis()
            x.SetRangeUser(-3.5,3.5)
            y=hist.GetYaxis()
            y.SetRangeUser(-3.2,3.2)
            hist.SetXTitle('#eta')
            hist.SetYTitle('#phi')

            signalR=ROOT.TEllipse(125.0,125.0,25.0,25.0)
            signalR.SetFillColorAlpha(0,0)
            signalR.SetLineColor(ROOT.kRed)
            signalR.Draw()

            for i in range(len(selbox)):
                selbox[i].Draw()
            CMSbox.Draw()
            extraTextBox.Draw()
            lumibox.Draw()


            canvas.SetRightMargin(0.18);
            canvas.Draw()
            canvas.SaveAs(prefixH+'/2D'+hname+'_'+tag+'.png')

## Et res as a split in eta

In [ ]:
for tt in   tagsToPlot:
    print(tt,legend[tt])

In [ ]:
prefixH=prefix+'/splitInEtaBins'
if not os.path.exists(prefixH):
    os.system('mkdir -p '+prefixH)
    
if True:
    plots=[]
    cPars=getCanvasParams(baseCanvas,lumi)
    rBin=1
    tag='dataR3Unpaked_postCalib'
    for hname in histStore[tag]:
        if 'L1EtResolutionInEta' not in hname:
            continue
        tagName=hname.replace('-','M').replace('.','p')
        etamin,etamax=hname.replace('L1EtResolutionInEta','').split('To')
        plot=getDefaultPlot(prefix=prefixH,name='split'+tagName,cPars=cPars) 
        plot.legendPosition = (0.16,0.31,0.5,0.45)
        plot.descPosition   = (0.21,0.75)
        plot.desc =[etamin+" < #eta < "+etamax]
        plot.yTitle = "a.u"  
        plot.xTitle = "E_{T}^{L1} / E_{T}^{"+varToChose+"}"
        plot.xRange = (0.3,1.3) 
        plot.yRange = (0.0,'auto') 
        plots.append(plot) 

        i=1
        for tt in   tagsToPlot :#['dataR3Unpaked_postCalib','run3MC','run3MCWithGenEt','run3MCWithGenEtVsGenEt']: 
            i+=1
            hist=histStore[tt][hname].Clone()
            hist.Rebin(rBin)
            pparams=getDefaultPlotParams(col=i,marker=22)
            pparams['Legend']=legend[tt]
            pparams['MarkerColor']= colours[tt]; pparams['LineColor'] = colours[tt]
            pparams['Options']='HIST pe'
            aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
            aplot.normalize = True ;aplot.scaleFactor = 1.0
            aplot.doFit=True ; aplot.drawFit=True
            plots[-1].addPlot(aplot)


    canvas = []
    for plot in plots:
        canvas.append(plot.plot())    

In [ ]:
if True:  # Inclusive Et barrel Ecap
    plots=[]
    cPars=getCanvasParams(baseCanvas,lumi)
    rBin=1
    
    plot=getDefaultPlot(prefix=prefix,name='resolutionET_Barrel'+varToChose,cPars=cPars) 
    plot.legendPosition = (0.16,0.70,0.66,0.88)
    plot.descPosition   = (0.25,0.65)
    plot.desc =[" "]
    plot.yTitle = "a.u"  
    plot.xTitle = "E_{T}^{L1} / E_{T}^{"+varToChose+"}"
    plot.xRange = (0.7,1.20) 
    plot.logx = False
    plot.yRange = (0.0,0.14) 
    plots.append(plot) 

    i=2
    for tag in tagsToPlot : #['run3MC','dataR3Unpaked_postCalib','data2018','run3MCWithGenEt','run3MCWithGenEtVsGenEt']:
        hname='EtVsdEtBareResBarrel'
        hist=histStore[tag][hname].Clone()
    #         hist.Rebin(rBin)
        pparams=getDefaultPlotParams(col=i,marker=22)
        pparams['Legend']='Barrel , '+legend[tag]
        pparams['MarkerColor']= colours[tag]; pparams['LineColor'] = colours[tag] ;i+=1;
        pparams['Options']='HIST pl'
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor =None
        aplot.doFit=True ; aplot.drawFit=False
        aplot.verbose= False
        plots[-1].addPlot(aplot)


    plot=getDefaultPlot(prefix=prefix,name='resolutionET_ECap',cPars=cPars) 
    plot.legendPosition = (0.2,0.72,0.72,0.90)
    plot.descPosition   = (0.65,0.75)
    plot.desc =[" "]
    plot.yTitle = "a.u"  
    plot.xTitle = "E_{T}^{L1} / E_{T}^{"+varToChose+"}"
    plot.xRange = (0.62,1.60) 
    plot.logx = False
    plot.yRange = (0.0,0.14) 
    plots.append(plot) 
    
    i=4
    hname='EtVsdEtBareResECap'
    for tag in tagsToPlot: #['run3MC','dataR3Unpaked_postCalib','data2018','run3MCWithGenEt','run3MCWithGenEtVsGenEt']:
        hist=histStore[tag][hname].Clone()
    #         hist.Rebin(rBin)
        pparams=getDefaultPlotParams(col=i,marker=22)
        pparams['Legend']='Endcap , '+legend[tag]
        pparams['MarkerColor']= colours[tag]; pparams['LineColor'] = colours[tag] ;i+=1;
        pparams['Options']='HIST pl'
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor =None
        aplot.doFit=True ; aplot.drawFit=False
        aplot.verbose= False
        plots[-1].addPlot(aplot)

    canvas = []
    for plot in plots:
        canvas.append(plot.plot())
    

In [ ]:
if True:  # Inclusive Et barrel Ecap
    plots=[]
    cPars=getCanvasParams(baseCanvas,lumi)
    rBin=1
    
    plot=getDefaultPlot(prefix=prefix,name='resolutionETGThan20_Barrel',cPars=cPars) 
    plot.legendPosition = (0.16,0.65,0.60,0.80)
    plot.descPosition   = (0.25,0.55)
    plot.desc =["L1 EG "," E_{T}^{"+varToChose+"} > 20 GeV"]
    plot.yTitle = "a.u"  
    plot.xTitle = "E_{T}^{L1} / E_{T}^{"+varToChose+"}"
    plot.xRange = (0.7,1.20) 
    plot.logx = False
    plot.yRange = (0.0,0.24) 
    plots.append(plot) 

    i=2
    for tag in tagsToPlot:#['run3MC','dataR3Unpaked_postCalib','run3MCWithGenEt','run3MCWithGenEtVsGenEt']:
        print(tag)
        hname='PtGThan20EtVsdEtBareResBarrel'
        hist=histStore[tag][hname].Clone()
    #         hist.Rebin(rBin)
        pparams=getDefaultPlotParams(col=i,marker=22)
        pparams['Legend']='Barrel , '+legend[tag]
        pparams['MarkerColor']= colours[tag]; pparams['LineColor'] = colours[tag] ;i+=1;
        pparams['Options']='HIST pl'
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor =None
        aplot.doFit=True ; aplot.drawFit=False
        aplot.verbose= False
        plots[-1].addPlot(aplot)


    plot=getDefaultPlot(prefix=prefix,name='resolutionETGthan20_ECap',cPars=cPars) 
    plot.legendPosition = (0.2,0.72,0.72,0.90)
    plot.descPosition   = (0.65,0.65)
    plot.desc =["L1 EG "," E_{T}^{offline} > 20 GeV"]
    plot.yTitle = "a.u"  
    plot.xTitle = "E_{T}^{L1} / E_{T}^{offline}"
    plot.xRange = (0.62,1.60) 
    plot.logx = False
    plot.yRange = (0.0,0.14) 
    plots.append(plot) 
    
    i=4
    hname='PtGThan20EtVsdEtBareResECap'
    for tag in tagsToPlot:#['run3MC','dataR3Unpaked_postCalib','run3MCWithGenEt','run3MCWithGenEtVsGenEt']:
        hist=histStore[tag][hname].Clone()
    #         hist.Rebin(rBin)
        pparams=getDefaultPlotParams(col=i,marker=22)
        pparams['Legend']='Endcap , '+legend[tag]
        pparams['MarkerColor']= colours[tag]; pparams['LineColor'] = colours[tag] ;i+=1;
        pparams['Options']='HIST pl'
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor =None
        aplot.doFit=True ; aplot.drawFit=False
        aplot.verbose= False
        plots[-1].addPlot(aplot)

    canvas = []
    for plot in plots:
        canvas.append(plot.plot())


In [ ]:
if True:  # Et vs dET barrel ECap
    plots=[]
    cPars=getCanvasParams(baseCanvas,lumi)
    rBin=1
    plot=getDefaultPlot(prefix=prefix,name='EtVsdEt_highResoRes_Barrel',cPars=cPars) 
    plot.legendPosition = (0.40,0.40,0.90,0.60)
    plot.descPosition   = (0.55,0.75)
    plot.desc =["L1 EG","Loose Offline Electron","Barrel Region"]
    plot.yTitle = "FWHM <E_{T}^{L1} / E_{T}^{"+varToChose+"}>"  
    plot.xTitle = "E_{T}^{"+varToChose+"}"
    plot.xRange = (5.0,100.0) 
    plot.logx = False
    plot.yRange = (0.0,0.35) 
    plots.append(plot) 

    i=2
    hname='EtVsdEt_highResoResBarrel'
    for tag in tagsToPlot : #['run3MC','dataR3Unpaked_postCalib','data2018','run3MCWithGenEt','run3MCWithGenEtVsGenEt']:
        hist=histStore[tag][hname].Clone()
        pparams=getDefaultPlotParams(col=i,marker=22)
        pparams['Legend']=legend[tag]
        pparams['MarkerColor']= colours[tag]; pparams['LineColor'] = colours[tag] ;i+=1;
        pparams['Options']='HIST pe'
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor =None
        aplot.doFit=False ; aplot.drawFit=False
        aplot.verbose= False
        plots[-1].addPlot(aplot)
    
    plot=getDefaultPlot(prefix=prefix,name='EtVsdEt_highResoRes_ECap',cPars=cPars) 
    plot.legendPosition = (0.40,0.40,0.90,0.60)
    plot.descPosition   = (0.55,0.75)
    plot.desc =["L1 EG","Loose Offline Electron","Endcap Region"]
    plot.yTitle = "FWHM <E_{T}^{L1} / E_{T}^{"+varToChose+"}>"  
    plot.xTitle = "E_{T}^{"+varToChose+"}"
    plot.xRange = (5.0,100.0) 
    plot.logx = False
    plot.yRange = (0.0,0.7) 
    plots.append(plot) 

    i=4
    hname='EtVsdEt_highResoResECap'
    for tag in tagsToPlot:# ['run3MC','dataR3Unpaked_postCalib','data2018','run3MCWithGenEt','run3MCWithGenEtVsGenEt']:
        hist=histStore[tag][hname].Clone()
        pparams=getDefaultPlotParams(col=i,marker=22)
        pparams['Legend']=legend[tag]
        pparams['MarkerColor']= colours[tag]; pparams['LineColor'] = colours[tag] ;i+=1;
        pparams['Options']='HIST pe'
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor =None
        aplot.doFit=False ; aplot.drawFit=False
        aplot.verbose= False
        plots[-1].addPlot(aplot)

    canvas = []
    for plot in plots:
        canvas.append(plot.plot())
    

In [ ]:
if True:  # Eta vs dET barrel ECap
    plots=[]
    cPars=getCanvasParams(baseCanvas,lumi)
    rBin=1
    plot=getDefaultPlot(prefix=prefix,name='EtaVsdEt_highResoRes',cPars=cPars) 
    plot.legendPosition = (0.25,0.41,0.68,0.60)
    plot.descPosition   = (0.25,0.75)
    plot.desc =["L1 EG","Loose Offline Electron","|#eta| <2.4"]
    plot.yTitle = "FWHM <E_{T}^{L1} / E_{T}^{"+varToChose+"}>"  
    plot.xTitle = "E_{T}^{"+varToChose+"}"
    plot.xRange = (0.0,2.6) 
    plot.logx = False
    plot.yRange = (0.0,0.5) 
    plots.append(plot) 

    i=2
    hname='EtaVsdEt_highResoResInclusive'
    for tag in tagsToPlot : #['run3MC','dataR3Unpaked_postCalib','data2018','run3MCWithGenEt','run3MCWithGenEtVsGenEt']:
        hist=histStore[tag][hname].Clone()
    #         hist.Rebin(rBin)
        pparams=getDefaultPlotParams(col=i,marker=22)
        pparams['Legend']=legend[tag]
        pparams['MarkerStyle']=8
        pparams['MarkerColor']= colours[tag]; pparams['LineColor'] = colours[tag]
        pparams['Options']='HIST pe'
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor =None
        aplot.doFit=False ; aplot.drawFit=False
        aplot.drawLegend=True
        aplot.verbose= False
        plots[-1].addPlot(aplot)

    canvas = []
    for plot in plots:
        canvas.append(plot.plot())
    

In [ ]:
if True:  # Eta and Phi Resolution
    plots=[]
    cPars=getCanvasParams(baseCanvas,lumi)
    rBin=1
    plot=getDefaultPlot(prefix=prefix,name='EtaResolution',cPars=cPars) 
    plot.legendPosition = (0.55,0.25,0.90,0.39)
    plot.descPosition   = (0.65,0.75)
    plot.desc =["L1 EG","Loose Offline Electron","|#eta| <2.4"]
    plot.yTitle = "a.u"  
    plot.xTitle = "#eta_{L1}-#eta_{"+varToChose+"}"
    plot.xRange = (-0.1,0.2) 
    plot.logx = False
    plot.yRange = (0.0,0.15) 
    plots.append(plot) 

    i=2
    hname='EtaVsdEtaBareResInclusive'
    for tag in tagsToPlot : #['run3MC','dataR3Unpaked_postCalib','data2018','run3MCWithGenEt','run3MCWithGenEtVsGenEt']:
        hist=histStore[tag][hname].Clone()
    #         hist.Rebin(rBin)
        pparams=getDefaultPlotParams(col=i,marker=22)
        pparams['Legend']=legend[tag]
        pparams['MarkerStyle']=8
        pparams['MarkerColor']= colours[tag]; pparams['LineColor'] = colours[tag]
        pparams['Options']='HIST pec'
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = True ;aplot.scaleFactor =None
        aplot.doFit=True ; aplot.drawFit=True
        aplot.drawLegend=True
        aplot.verbose= False
        plots[-1].addPlot(aplot)

    canvas = []
    for plot in plots:
        canvas.append(plot.plot())

        
if True:  # Eta and Phi Resolution
    plots=[]
    cPars=getCanvasParams(baseCanvas,lumi)
    rBin=1
    plot=getDefaultPlot(prefix=prefix,name='PhiResolution',cPars=cPars) 
    plot.legendPosition = (0.55,0.25,0.90,0.39)
    plot.descPosition   = (0.65,0.75)
    plot.desc =["L1 EG","Loose Offline Electron","|#eta| <2.4"]
    plot.yTitle = "a.u"  
    plot.xTitle = "#phi_{L1}-#phi_{offline}"
    plot.xRange = (-0.15,0.25) 
    plot.logx = False
    plot.yRange = (0.0,0.15) 
    plots.append(plot) 

    i=2
    hname='PhiVsdPhiBareResInclusive'
    for tag in tagsToPlot : #['run3MC','dataR3Unpaked_postCalib','data2018','run3MCWithGenEt','run3MCWithGenEtVsGenEt']:
        hist=histStore[tag][hname].Clone()
    #         hist.Rebin(rBin)
        pparams=getDefaultPlotParams(col=i,marker=22)
        pparams['Legend']=legend[tag]
        pparams['MarkerStyle']=8
        pparams['MarkerColor']= colours[tag]; pparams['LineColor'] = colours[tag]
        pparams['Options']='HIST pec'
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = True ;aplot.scaleFactor =None
        aplot.doFit=True ; aplot.drawFit=True
        aplot.drawLegend=True
        aplot.verbose= False
        plots[-1].addPlot(aplot)

    canvas = []
    for plot in plots:
        canvas.append(plot.plot())
            

In [ ]:
2/0

## Offline Scales

In [ ]:
f=ROOT.TFile('data/oct25/TagAndProbe_DataRun3_2022Eres_defLUT_newIso_vsSCEt.root')

In [ ]:
offlineScaleStore=getTheObjectsFromFile(f)['0']

### PF Et vs GEN Et

In [ ]:
if False:  # Eta vs dET barrel ECap
    plots=[]
    cPars=getCanvasParams('Run3MC',lumi)
    rBin=1
    plot=getDefaultPlot(prefix=prefix,name='GenEtVsPFEtoveGenEtScale',cPars=cPars) 
    plot.legendPosition = (0.45,0.61,0.88,0.70)
    plot.descPosition   = (0.45,0.85)
    plot.desc =["L1 EG","Loose Offline Electron","|#eta| <2.4"]
    plot.yTitle = "E_{T}^{Offline} / E_{T}^{GEN}"  
    plot.xTitle = "E_{T}^{GEN}"
    plot.xRange = (0.0,150.0) 
    plot.logx = False
    plot.yRange = (0.6,1.8) 
    plots.append(plot) 

    i=2
    hname='GenEtVsPFEtoveGenEtScaleBarrel'
    hist=offlineScaleStore[hname].Clone()
#         hist.Rebin(rBin)
    pparams=getDefaultPlotParams(col=i,marker=22)
    pparams['Legend']='E_{T}^{Offline} / E_{T}^{GEN} , Barrel'
    pparams['MarkerStyle']=8
    pparams['MarkerColor']= ROOT.kBlue; pparams['LineColor'] = ROOT.kBlue
    pparams['Options']='HIST pe'
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor =None
    aplot.doFit=False ; aplot.drawFit=False
    aplot.drawLegend=True
    aplot.verbose= False
    plots[-1].addPlot(aplot)
    
    hname='GenEtVsPFEtoveGenEtScaleECap'
    hist=offlineScaleStore[hname].Clone()
#         hist.Rebin(rBin)
    pparams=getDefaultPlotParams(col=i,marker=22)
    pparams['Legend']='E_{T}^{Offline} / E_{T}^{GEN} , Endcap'
    pparams['MarkerStyle']=8
    pparams['MarkerColor']= ROOT.kRed; pparams['LineColor'] = ROOT.kRed
    pparams['Options']='HIST pe'
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor =None
    aplot.doFit=False ; aplot.drawFit=False
    aplot.drawLegend=True
    aplot.verbose= False
    plots[-1].addPlot(aplot)

    canvas = []
    for plot in plots:
        canvas.append(plot.plot())
    

## SC Et vs PFEt

In [ ]:
if True:  # Eta vs dET barrel ECap
    plots=[]
    cPars=getCanvasParams('Run3DataX',lumi)
    rBin=1
    plot=getDefaultPlot(prefix=prefix,name='PFEtVsSCEtovePFEtScale',cPars=cPars) 
    plot.legendPosition = (0.45,0.35,0.88,0.50)
    plot.descPosition   = (0.45,0.85)
    plot.desc =["L1 EG","Loose Offline Electron","|#eta| <2.4"]
    plot.yTitle = "E_{T}^{SC} / E_{T}^{Offline}"  
    plot.xTitle = "E_{T}^{Offline}"
    plot.xRange = (0.0,150.0) 
    plot.logx = False
    plot.yRange = (0.6,1.2) 
    plots.append(plot) 

    i=2
    hname='PFEtVsSCEtovePFEtScaleBarrel'
    hist=offlineScaleStore[hname].Clone()
#         hist.Rebin(rBin)
    pparams=getDefaultPlotParams(col=i,marker=22)
    pparams['Legend']='E_{T}^{SC} / E_{T}^{Offline} , Barrel'
    pparams['MarkerStyle']=8
    pparams['MarkerColor']= ROOT.kBlue; pparams['LineColor'] = ROOT.kBlue
    pparams['Options']='HIST pe'
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor =None
    aplot.doFit=False ; aplot.drawFit=False
    aplot.drawLePFd=True
    aplot.verbose= False
    plots[-1].addPlot(aplot)
    
    hname='PFEtVsSCEtovePFEtScaleECap'
    hist=offlineScaleStore[hname].Clone()
#         hist.Rebin(rBin)
    pparams=getDefaultPlotParams(col=i,marker=22)
    pparams['Legend']='E_{T}^{SC} / E_{T}^{Offline} , Endcap'
    pparams['MarkerStyle']=8
    pparams['MarkerColor']= ROOT.kRed; pparams['LineColor'] = ROOT.kRed
    pparams['Options']='HIST pe'
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor =None
    aplot.doFit=False ; aplot.drawFit=False
    aplot.drawLePFd=True
    aplot.verbose= False
    plots[-1].addPlot(aplot)

    canvas = []
    for plot in plots:
        canvas.append(plot.plot())
    

## SCEt vs GENEt

In [ ]:
if False:  # Eta vs dET barrel ECap
    plots=[]
    cPars=getCanvasParams('Run3MC',lumi)
    rBin=1
    plot=getDefaultPlot(prefix=prefix,name='GenEtVsSCEtoveGenEtScale',cPars=cPars) 
    plot.lePFdPosition = (0.45,0.61,0.88,0.70)
    plot.descPosition   = (0.45,0.85)
    plot.desc =["L1 EG","Loose Offline Electron","|#eta| <2.4"]
    plot.yTitle = "E_{T}^{SC} / E_{T}^{GEN}"  
    plot.xTitle = "E_{T}^{GEN}"
    plot.xRange = (0.0,150.0) 
    plot.logx = False
    plot.yRange = (0.6,1.2) 
    plots.append(plot) 

    i=2
    hname='GenEtVsSCEtoveGenEtScaleBarrel'
    hist=offlineScaleStore[hname].Clone()
#         hist.Rebin(rBin)
    pparams=getDefaultPlotParams(col=i,marker=22)
    pparams['Legend']='E_{T}^{SC} / E_{T}^{GEN} , Barrel'
    pparams['MarkerStyle']=8
    pparams['MarkerColor']= ROOT.kBlue; pparams['LineColor'] = ROOT.kBlue
    pparams['Options']='HIST pe'
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor =None
    aplot.doFit=False ; aplot.drawFit=False
    aplot.drawLePFd=True
    aplot.verbose= False
    plots[-1].addPlot(aplot)
    
    hname='GenEtVsSCEtoveGenEtScaleECap'
    hist=offlineScaleStore[hname].Clone()
#         hist.Rebin(rBin)
    pparams=getDefaultPlotParams(col=i,marker=22)
    pparams['Legend']='E_{T}^{SC} / E_{T}^{GEN} , Endcap'
    pparams['MarkerStyle']=8
    pparams['MarkerColor']= ROOT.kRed; pparams['LineColor'] = ROOT.kRed
    pparams['Options']='HIST pe'
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor =None
    aplot.doFit=False ; aplot.drawFit=False
    aplot.drawLePFd=True
    aplot.verbose= False
    plots[-1].addPlot(aplot)

    canvas = []
    for plot in plots:
        canvas.append(plot.plot())
    

## Rate Plotter

In [ ]:
prefixH=prefix+'/rates'

if not os.path.exists(prefixH):
    os.system('mkdir -p '+prefixH)

In [ ]:
fnamesForRate={
    
'run357815GenEtCalibInOldIso': 'data/oct14/ratesFromSweta/HistgramFile_Rate_Run3_2022D_357815_genEt_oldIso_HCalTP.root',
'run357815HybEtCalibInNewIso': 'data/oct14/ratesFromSweta/HistgramFile_Rate_Run3_2022D_357815_Hybrid_newIso_HCalTP.root',
'run357815HybEtCalibInOldIso': 'data/oct14/ratesFromSweta/HistgramFile_Rate_Run3_2022D_357815_Hybrid_oldIso_HCalTP.root',
 'run357815SCEtCalibInOldIso': 'data/oct14/ratesFromSweta/HistgramFile_Rate_Run3_2022D_357815_SC_oldIso_HCalTP.root',
}

In [ ]:
legendsForRate={
'run357815GenEtCalibInOldIso': 'E_{T}^{GEN} based Calieration',
'run357815HybEtCalibInNewIso': 'Hybrid Calibration',
'run357815HybEtCalibInOldIso': 'Hybrid Calibration',
 'run357815SCEtCalibInOldIso': 'Default Calibration',
}

colorsForRate={
'run357815GenEtCalibInOldIso': ROOT.kRed ,
'run357815HybEtCalibInNewIso': ROOT.kBlue ,
'run357815HybEtCalibInOldIso': ROOT.kGreen+2 ,
 'run357815SCEtCalibInOldIso': ROOT.kRed ,
}

tagsToPlotRates=[
    'run357815HybEtCalibInNewIso',
    'run357815SCEtCalibInOldIso'
]

In [ ]:
# histRatesStore={}
# histRatesFileStore={}
# for tag in fnamesForRate:
#     histRatesFileStore[tag]=ROOT.TFile(fnamesForRate[tag],'READ')
#     print("loading  tag ",tag)
#     histRatesStore[tag]=getTheObjectsFromFile(histRatesFileStore[tag])

In [ ]:
if False:  # SingleEGSio
    plots=[]
    cPars=getCanvasParams('Run3DataZB','Run 357815')
    rBin=1
    plot=getDefaultPlot(prefix=prefixH,name='SingleEG_rate_inclusive',cPars=cPars) 
    plot.lePFdPosition = (0.45,0.61,0.88,0.70)
    plot.descPosition   = (0.45,0.85)
    plot.desc =["L1 EG","Zero-Bias","Single EG Rate"]
    plot.yTitle = "Rate [kHz]"  
    plot.xTitle = "E_{T}^{L1}"
    plot.xRange = (20.0,50.0) 
    plot.logx = False
    plot.yRange = (1.0,50.0) 
    plots.append(plot) 

    i=2
    hname='SingleEG_rate_inclusive'
    for tag in tagsToPlotRates:

        hist=histRatesStore[tag][hname].Clone()
    #         hist.Rebin(rBin)
        pparams=getDefaultPlotParams(col=i,marker=22)
        pparams['Legend']=legendsForRate[tag]
        pparams['MarkerStyle']=8
        pparams['MarkerColor']= colorsForRate[tag]; pparams['LineColor'] =colorsForRate[tag]
        pparams['Options']='HIST pe'
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor =None
        aplot.doFit=False ; aplot.drawFit=False
        aplot.drawLePFd=True
        aplot.verbose= False
        plots[-1].addPlot(aplot)
 
    canvas = []
    for plot in plots:
        canvas.append(plot.plot())
    

In [ ]:
if False:  # Double EG Inclusive
    plots=[]
    cPars=getCanvasParams('Run3DataZB','Run 357815')
    rBin=1
    plot=getDefaultPlot(prefix=prefixH,name='DoubleEG_rate_inclusive',cPars=cPars) 
    plot.lePFdPosition = (0.45,0.61,0.88,0.70)
    plot.descPosition   = (0.45,0.85)
    plot.desc =["L1 EG","Zero-Bias","Double EG Rate"]
    plot.yTitle = "Rate [kHz]"  
    plot.xTitle = "E_{T}^{L1}"
    plot.xRange = (5.0,20.0) 
    plot.logx = False
    plot.yRange = (0.0,50.0) 
    plots.append(plot) 

    i=2
    hname='DoubleEG_rate_inclusive'
    for tag in tagsToPlotRates:

        hist=histRatesStore[tag][hname].Clone()
    #         hist.Rebin(rBin)
        pparams=getDefaultPlotParams(col=i,marker=22)
        pparams['Legend']=legendsForRate[tag]
        pparams['MarkerStyle']=8
        pparams['MarkerColor']= colorsForRate[tag]; pparams['LineColor'] =colorsForRate[tag]
        pparams['Options']='HIST pe'
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor =None
        aplot.doFit=False ; aplot.drawFit=False
        aplot.drawLePFd=True
        aplot.verbose= False
        plots[-1].addPlot(aplot)
 
    canvas = []
    for plot in plots:
        canvas.append(plot.plot())
    

In [ ]:
histRatesStore.keys()

In [ ]:
tag='run357815HybEtCalibInNewIso'
hname='DoubleEG_rate_inclusive'
hname='SingleEG_rate_LooseIso'
hname='DoubleEG_rate_LooseIso'
hname='SingleEG_rate_TightIso'
hname='SingleEG_rate_inclusive'

for i in range(histRatesStore[tag][hname].GetNbinsX()):
    x=histRatesStore[tag][hname].GetBinCenter(i)
    y=histRatesStore[tag][hname].GetBinContent(i)
    print(tag,hname," | i = ",i,
          " | x = ",x,
          " | y = ",y)